# Загрузка библиотек. Подготовка данных

In [18]:
import csv
import os
from itertools import combinations
import pandas as pd
from collections import Counter
from sklearn.metrics import jaccard_score
import glob

In [8]:
# Загрузка файла
file_path = "../output/for_gephi/bimodal_edges.csv"

In [4]:
# Путь к папке для сохранения результата
output_dir = os.path.join("..", "output", "stats_tables")
os.makedirs(output_dir, exist_ok=True)

# Определяем гапаксы для пар сборников

In [10]:
# Имя итогового CSV
output_path = os.path.join(output_dir, "targets_with_2_occurrences.csv")

# Чтение данных
df = pd.read_csv(file_path)

# Находим значения Target, которые встречаются ровно 2 раза
target_counts = df["Target"].value_counts()
targets_with_2_occurrences = target_counts[target_counts == 2].index

# Фильтруем исходные данные
filtered_df = df[df["Target"].isin(targets_with_2_occurrences)]

# Группировка по Target и сбор уникальных Source
grouped = (
    filtered_df
    .groupby("Target")["Source"]
    .unique()
    .reset_index()
)

# Преобразуем список Source в строку
grouped["Source"] = grouped["Source"].apply(lambda sources: ", ".join(sorted(sources)))

# Сохраняем результат
grouped.to_csv(output_path, index=False, sep=";")

print(f"Результат сохранён: {output_path}")


Результат сохранён: ../output/stats_tables/targets_with_2_occurrences.csv


# Считаем количество гапаксов для каждой пары

In [16]:
# Загрузка файла
file_path = "../data/csv/targets_with_sources.csv"

# Папка для сохранения результата
output_dir = os.path.join("..", "output", "stats_tables")
os.makedirs(output_dir, exist_ok=True)

# Путь к выходному файлу
output_path = os.path.join(output_dir, "source_pair_target_counts.csv")

# Читаем CSV (разделитель — запятая)
df = pd.read_csv(file_path)

# Группируем по Target и получаем список Source для каждого
target_groups = df.groupby("Target")["Source"].unique()

# Выбираем только те Targets, которые встречаются в ровно двух источниках
target_pairs = target_groups[target_groups.apply(len) == 2]

# Считаем, сколько раз каждая пара источников делит общий Target
pair_counter = Counter()

for sources in target_pairs:
    pair = tuple(sorted(sources))
    pair_counter[pair] += 1

# Формируем DataFrame
result_df = pd.DataFrame(pair_counter.items(), columns=["Source_Pair", "Target_Count"])

# Разбиваем пары на отдельные колонки
result_df[["Source_1", "Source_2"]] = pd.DataFrame(result_df["Source_Pair"].tolist(), index=result_df.index)
result_df.drop(columns=["Source_Pair"], inplace=True)

# Сохраняем результат
result_df.to_csv(output_path, index=False)

print(f"✅ Результат сохранён в файл: {output_path}")


✅ Результат сохранён в файл: ../output/stats_tables/source_pair_target_counts.csv


# Определяем коэффициент Жаккара между парами сборников

In [20]:
# Путь к папке с файлами
lemmas_dir = os.path.join("..", "data", "lemmas")

# Папка для результата
output_dir = os.path.join("..", "output", "stats_tables")
os.makedirs(output_dir, exist_ok=True)

# Имя итогового CSV
output_path = os.path.join(output_dir, "jaccard_similarity_matrix.csv")

# Собираем все файлы cleaned_*.csv
files = glob.glob(os.path.join(lemmas_dir, "cleaned_*.csv"))

# Словарь: имя файла → множество лемм
file_lemmas = {}

# Чтение и извлечение единственной колонки из каждого файла
for file_path in files:
    try:
        df = pd.read_csv(file_path, header=None, names=["lemma"])
        lemmas = set(df["lemma"].dropna().astype(str))
        file_label = os.path.basename(file_path).replace("cleaned_", "")
        file_lemmas[file_label] = lemmas
    except Exception as e:
        print(f"Ошибка в {file_path}: {e}")

# Вычисление коэффициента Жаккара для всех пар
results = []
for (file1, lemmas1), (file2, lemmas2) in combinations(file_lemmas.items(), 2):
    intersection = len(lemmas1 & lemmas2)
    union = len(lemmas1 | lemmas2)
    jaccard = intersection / union if union > 0 else 0
    results.append((file1, file2, round(jaccard, 4)))

# Преобразуем в DataFrame
jaccard_df = pd.DataFrame(results, columns=["File_1", "File_2", "Jaccard_Similarity"])

# Сохраняем
jaccard_df.to_csv(output_path, index=False)

print(f"Коэффициенты Жаккара сохранены в: {output_path}")


Коэффициенты Жаккара сохранены в: ../output/stats_tables/jaccard_similarity_matrix.csv
